In [1]:
from dotenv import load_dotenv
import os
from src.utils import Database

load_dotenv()

db_config = {
    "dbname": "fpkg",
    "user": os.environ.get("PGUSER"),
    "password": os.environ.get("PGPASSWORD"),
    "host": "localhost",
    "port": "5431"
}

db = Database(db_config)

Successfully initialized database and OpenAI embeddings


In [2]:
drug_of_interest = db.vsearch_drug(drug="Zoloft 50mg")

found 5 results

Top result:
    Name: ZOLOFT 50MG GELULE
    ID: 13664



In [3]:
# Finding all of the generics of Zoloft in the db thru graph search
if drug_of_interest and "id" in drug_of_interest:
    id_of_drug = drug_of_interest["id"]
    generics = db.find_generics(id_of_drug)
else:
    print("No drug found or drug_of_interest is None")

Found 12 results:
SERTRALINE ALM 50MG GELULE
SERTRALINE ALT 50MG GELULE
SERTRALINE ARL 50MG GELULE
SERTRALINE BGA 50MG GELULE
SERTRALINE CRP 50MG GELULE
SERTRALINE EG 50MG GELULE
SERTRALINE EVO 50MG GELULE
SERTRALINE SDZ 50MG GELULE
SERTRALINE TVC 50MG GELULE
SERTRALINE VIA 50MG GELULE
SERTRALINE ZEN 50MG GELULE
SERTRALINE ZYF 50MG GELULE
